In [ ]:
import polars as pl
import numpy as np

from plotly import graph_objects as go, io as pio

pio.templates.default = "plotly_dark"

df = (
    pl.read_parquet("../data/bench_20.parquet")
    .filter(
        pl.col("example").is_in(["Sphere (1d)", "Sphere (2d)"]).not_(),
        # pl.col("method").is_in(["OptDummy"]).not_(),
    )
    .with_columns(pl.col("ls_method").str.replace("exact", "naive"))
)

df.glimpse()


examples = df["example"].unique().sort().to_list()
methods = df["method"].unique().sort().to_list()
ls_methods = df["ls_method"].unique().sort().to_list()

print("examples   :", examples)
print("methods    :", methods)
print("ls_methods :", ls_methods)


![](../Funs.png)


## Error distribution

How likely are different errors?

NOTE: Linear histogram + logarithmic axis -> a bit tricky?


In [ ]:
GROUPBY = ["method", "ls_method"]
# GROUPBY = ["method"]
# GROUPBY = ["ls_method"]
cmap = dict(
    zip(
        methods if "method" in GROUPBY else ls_methods,
        ["#636EFA", "#EF553B", "#00CC96", "#FFA15A", "#19D3F3", "#B6E880", "#FECB52"],
    )
)

for ex in examples:
    fig = go.Figure(
        layout=go.Layout(
            width=800,
            height=400,
            xaxis_type="log",
            title=f"{ex} (group by {', '.join(GROUPBY)})",
            xaxis_title="error",
            yaxis_title="density",
            hoverlabel=dict(font_size=16),
            # showlegend=False,
        ),
    )

    data = df.filter(example=ex)

    for grname, df_g in data.group_by(GROUPBY, maintain_order=True):
        # errs = np.stack(df_g["errors"].to_list(), axis=0).mean(0)
        errs = np.stack(df_g["errors"].to_list()).flatten()

        # print(f"{str(grname):<30}  :{errs.shape=}, {errs.min()=}")
        h, b = np.histogram(errs, 10)
        cent = (b[1:] + b[:-1]) / 2.0
        # cent = b[:-1]

        # print(f"{h=}")
        # print(f"{cent=}")
        fig.add_trace(
            go.Scatter(
                x=cent + 1e-10,
                y=h,
                name=".".join(str(k) for k in grname),
                text=".".join(str(k) for k in grname),
                line_color=cmap[grname[0]],
                mode="markers+lines",
                line_width=2,
                marker_size=3,
                hoverinfo="text",
            )
        )

    fig.show()

![](../Happrox.png)


## Mean error per method

How does the Method and LS Method choice impact mean error?


In [ ]:
for ex in examples:
    data = df.filter(example=ex)

    reshaped = data.select("method", "ls_method", "e_avg").pivot(
        values="e_avg", index="method", on="ls_method"
    )
    mat = reshaped.drop("method")[1:-1].to_numpy()
    mets = reshaped["method"][1:-1].to_list()
    lsmets = reshaped.drop("method")[1:-1].columns
    print(lsmets)
    assert mat.ndim == 2
    # print(mat.shape)

    fig = go.Figure(
        go.Heatmap(z=(mat), y=mets),
        layout=go.Layout(width=400, height=400, xaxis_type="log", title=ex),
    )

    fig.show()